In [1]:
from pyarrow.parquet import ParquetFile
from torch.utils.data import *
import pyarrow as pa
import pyarrow.parquet as pq
import glob,os,re,time
import numpy as np
from multiprocessing import Pool
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplhep as hep
import torch
import gc
import time, sys, os
plt.style.use([hep.style.ROOT, hep.style.firamath])
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print( f"device  : {device}" )

device  : cuda:0


In [2]:
# ls -lhtr /pscratch/sd/b/bbbam/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To17p2_dataset_2_unbaised_v2_train

In [3]:
hcal_scale  = torch.tensor(0.02)
ecal_scale  = torch.tensor(0.02)
pt_scale    = torch.tensor(0.1)
dz_scale    = torch.tensor(0.0005)
d0_scale    = torch.tensor(0.002)
pix_scale    = torch.tensor(0.1)
tracker_scale   = torch.tensor(0.05)
indices_ = [0,1,2,3,4,5,6,7,8,9,10,11,12]
indices      = torch.tensor(indices_)

In [4]:
def transform_y(y):
    return y/m0_scale

def inv_transform(y):
    return y*m0_scale


class ParquetDataset(Dataset):
    def __init__(self, filename, label):
        self.parquet = pq.ParquetFile(filename)
        self.cols = ['X_jet.list.item.list.item.list.item','am']
        self.label = label
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        data['X_jet'] = torch.tensor(data['X_jet'][0], dtype=torch.float16)
        data['am'] = torch.tensor(data['am'], dtype=torch.float16)
        return (data['X_jet'],data['am'])
    def __len__(self):
        return self.parquet.num_row_groups

def zs(data, indices):
    #High Value Suppression
    data[:, 0, :, :][data[:, 0, :, :] > 100] = 0. # Track pt
    data[:, 1, :, :][abs(data[:, 1, :, :]) > 2000] = 0 # Track dz
    data[:, 2, :, :][abs(data[:, 2, :, :]) > 500] = 0 # Track d0
    data[:, 3, :, :][data[:, 3, :, :] > 50] = 0. # Ecak
    data[:, 4, :, :][data[:, 4, :, :] > 50] = 0. # Hcal
    data[:, 5, :, :][data[:, 5, :, :] > 10] = 0. #pix_1
    data[:, 6, :, :][data[:, 6, :, :] > 10] = 0. #pix_2
    data[:, 7, :, :][data[:, 7, :, :] > 10] = 0. #pix_3
    data[:, 8, :, :][data[:, 8, :, :] > 10] = 0. #pix_4
    data[:, 9, :, :][data[:, 9, :, :] > 20] = 0. #Tib_1
    data[:, 10, :, :][data[:, 10, :, :] > 20] = 0. #Tib_2
    data[:, 11, :, :][data[:, 11, :, :] > 20] = 0. #Tob_1
    data[:, 12, :, :][data[:, 12, :, :] > 20] = 0. #Tob_2

    # Preprocessing
    data[:, 0, :, :] = pt_scale * data[:, 0, :, :]  #Track pT
    data[:, 1, :, :] = dz_scale * data[:, 1, :, :] #Track dZ
    data[:, 2, :, :] = d0_scale * data[:, 2, :, :] #Track d0
    data[:, 3, :, :] = ecal_scale * data[:, 3, :, :] #ECAL
    data[:, 4, :, :] = hcal_scale * data[:, 4, :, :] #HCAL
    data[:, 5, :, :] = pix_scale * data[:, 5, :, :] #pix_1
    data[:, 6, :, :] = pix_scale * data[:, 6, :, :] #pix_2
    data[:, 7, :, :] = pix_scale * data[:, 7, :, :] #pix_3
    data[:, 8, :, :] = pix_scale * data[:, 8, :, :] #pix_4
    data[:, 9, :, :] = tracker_scale * data[:, 9, :, :] #Tib_1
    data[:, 10, :, :] = tracker_scale * data[:, 10, :, :] #Tib_2
    data[:, 11, :, :] = tracker_scale * data[:, 11, :, :] #Tob_1
    data[:, 12, :, :] = tracker_scale * data[:, 12, :, :] #Tob_2

    data = torch.index_select(data, 1, indices.to(device))
    return data


In [5]:
import torch.nn as nn
import torch.hub

class ModifiedResNet(nn.Module):
    def __init__(self, resnet_ = 'resnet50', input_channels=13, out_channels=1):
        super(ModifiedResNet, self).__init__()
        # Load the ResNet-34 architecture without pre-trained weights
        self.resnet = torch.hub.load('pytorch/vision:v0.10.0', resnet_, pretrained=False)

        # Modify the first layer to accept the specified number of input channels
        self.resnet.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Modify the fully connected layer for regression (assuming 1 output for mass regression)
        out_feather = {'resnet18': 512, 'resnet34': 512, 'resnet50': 2048,'resnet101':2048, 'resnet152':2048}.get(resnet_, None)
        self.resnet.fc = nn.Linear(out_feather, out_channels)

    def forward(self, x):
        return self.resnet(x)

In [12]:
# ! ls -l /pscratch/sd/b/bbbam/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To17p2_dataset_2_unbaised_v2_train/

In [6]:
## -------Excute this line only save certian data from parquet to pkl 
class ParquetDataset(Dataset):
    def __init__(self, filename, label):
        self.parquet = pq.ParquetFile(filename)
        self.cols = ['X_jet.list.item.list.item.list.item','am']
    def __getitem__(self, index):
        data = self.parquet.read_row_group(index, columns=self.cols).to_pydict()
        data['X_jet'] = torch.tensor(data['X_jet'][0], dtype=torch.float32)
        data['am'] = torch.tensor(data['am'], dtype=torch.float32)
        return (data['X_jet'],data['am'])
    def __len__(self):
        return self.parquet.num_row_groups






BATCH_SIZE = 50

train_decays = glob.glob('/pscratch/sd/b/bbbam/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To17p2_dataset_2_unbaised_v2_train/IMG_aToTauTau_Hadronic_tauDR0p4_m1p2To3p6_dataset_2_unbaised_unphysical_0000_train.parquet')
dset_train = ConcatDataset([ParquetDataset('%s'%d,i) for i,d in enumerate(train_decays)])
n_eve = len(dset_train)
print("Total events.  ", n_eve)

n_train = ( len(dset_train) // BATCH_SIZE ) * BATCH_SIZE
train_sampler = RandomSampler(dset_train, replacement=True, num_samples=n_train)
train_loader  = DataLoader(dataset=dset_train, batch_size=BATCH_SIZE, num_workers=32, pin_memory=True , sampler=train_sampler)
data_size = sys.getsizeof(train_loader)
print("data_size. :  ",data_size)                          
print("train_sampler",len(train_sampler))  
print("train_loader",len(train_loader))




Total events.   92081
data_size. :   48
train_sampler 92050
train_loader 1841


In [7]:
import cProfile

In [8]:
pr = cProfile.Profile()
pr.enable()

start1_time = time.time()
for i, data in enumerate(train_loader):
    start_time = time.time()
    x, am = data[0].to(device), data[1].to(device)
    loading_time = time.time()
    torch.cuda.synchronize() # Synchronize to measure accurate GPU time
    x = zs(x, indices)
    end_time = time.time()
    # print('loading time :', loading_time - start_time,'   calculation time  :', end_time - start_time)
    if i % 10 == 0:
        print(f"batch loaded: {i}")
        print("X.shape", x.shape)

    if i == 1:
        break

torch.cuda.synchronize()  # Synchronize before measuring end time
end1_time = time.time()
# print("total time. :", end1_time-start1_time)
pr.disable()
pr.print_stats(sort='cumtime')  


batch loaded: 0
X.shape torch.Size([50, 13, 125, 125])
         24034 function calls (24028 primitive calls) in 9.311 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    9.311    1.862 interactiveshell.py:3472(run_code)
      8/5    0.000    0.000    9.311    1.862 {built-in method builtins.exec}
        1    0.001    0.001    9.311    9.311 2371988557.py:5(<module>)
      703    6.535    0.009    6.535    0.009 {method 'acquire' of '_thread.lock' objects}
       85    0.000    0.000    6.512    0.077 threading.py:270(wait)
        2    0.000    0.000    5.934    2.967 dataloader.py:676(__next__)
        2    0.000    0.000    5.932    2.966 dataloader.py:1329(_next_data)
       18    0.000    0.000    5.932    0.330 dataloader.py:1296(_get_data)
       18    0.000    0.000    5.932    0.330 dataloader.py:1150(_try_get_data)
       18    0.000    0.000    5.932    0.330 queue.py:153(get)
       

In [48]:
# device ='cuda:0'
device ='cpu'
start_time = time.time()
for i, data in enumerate(train_loader):
    am, pt = data[0].to(device), data[1].to(device)
    x = am * pt *am*pt
    if i==0: print(f"len x : {len(x)}")
    if i == 1000:
        break
end_time = time.time()  
print('totoal time. :', end_time-start_time)

len x : 100
totoal time. : 4.906213998794556


In [4]:
!pwd

/global/u1/b/bbbam/H_AA_CLuster_jupyter_notebooks


In [ ]:
%run /global/u1/b/bbbam/ML_training_script/mass_trainer_without_iphi_ieta.py /global/u1/b/bbbam/ML_training_script/PL_mass_config.yaml 

>>>> Channels used for this model >>>  Tracks_pt | Tracks_dZSig | Tracks_d0Sig | ECAL_energy | HBHE_energy | Pix_1 | Pix_2 | Pix_3 | Pix_4 | Tib_1 | Tib_2 | Tob_1 | Tob_2
>>>> Device Used >>>  cuda:0
>> Number of samples in dataset: Train: 5296937 + Val: 597421
>> Number of samples used for training: Train: 5280000 + Val: 570000


Using cache found in /global/homes/b/bbbam/.cache/torch/hub/pytorch_vision_v0.10.0


>> Training <<<<<<<<
>> Epoch 1 <<<<<<<<
